In [7]:
from custompackage.load_data import *
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [8]:
bs = 256
weighting = 'paired'
trials = 10
ds_set = ['emnist']
classes = 26

# Initialize for record keeping
paired_test = np.zeros((len(ds_set), trials, classes, classes))

# For each 10-class dataset
for k, ds in enumerate(ds_set):
    # Go through each class
    for t1 in range(10, 10 + classes):
        # and pair it with every other class
        for t2 in range(t1 + 1, 10 + classes):

            for m in range(trials):
                # Load the binary classification dataloaders
                trainloaders, validloaders, testloader = dataset_weighted_split_all(bs, t1, t2, weighting, trials, ds)

                # Assign entirety of the datasets within each dataloader to a variable
                X_train = trainloaders[0].dataset.tensors[0]
                y_train = trainloaders[0].dataset.tensors[1]
                X_test = testloader.dataset.tensors[0]
                y_test = testloader.dataset.tensors[1]

                # initialize lda
                lda = LinearDiscriminantAnalysis()

                # fit to images, labels
                lda.fit(X_train, y_train)

                # see accuracy for validation set
                score_test = lda.score(X_test, y_test)

                print(ds, m, t1, t2, score_test)

                # record keeping

                paired_test[k, m, t1 - 10, t2 - 10] = score_test

                np.save('results/confused_pairs_emnist.npy', paired_test)

  0%|          | 0/561753746 [00:00<?, ?it/s]

Extracting ./data\EMNIST\raw\gzip.zip to ./data\EMNIST\raw
emnist 0 10 11 0.8508771929824561
emnist 1 10 11 0.8304093567251462
emnist 2 10 11 0.8608187134502924
emnist 3 10 11 0.8461988304093567
emnist 4 10 11 0.8502923976608188
emnist 5 10 11 0.8450292397660819
emnist 6 10 11 0.8391812865497076
emnist 7 10 11 0.8467836257309942
emnist 8 10 11 0.8497076023391813
emnist 9 10 11 0.8473684210526315
emnist 0 10 12 0.9453766511960014
emnist 1 10 12 0.9550160656908248
emnist 2 10 12 0.952873973580864
emnist 3 10 12 0.9432345590860407
emnist 4 10 12 0.947161727954302
emnist 5 10 12 0.9407354516244198
emnist 6 10 12 0.9485897893609425
emnist 7 10 12 0.9418064976794002
emnist 8 10 12 0.9421635130310604
emnist 9 10 12 0.9450196358443413
emnist 0 10 13 0.9038565996740902
emnist 1 10 13 0.9033134166214014
emnist 2 10 13 0.8832156436719174
emnist 3 10 13 0.8902770233568713
emnist 4 10 13 0.9092884302009777
emnist 5 10 13 0.9005975013579577
emnist 6 10 13 0.888647474198805
emnist 7 10 13 0.883215643

KeyboardInterrupt: 